In [11]:
import pandas as pd
from db_utils import get_db_engine

In [5]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\Sleep_health_and_lifestyle_dataset.csv'
file4 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)

df1['id_person'] = range(1, len(df1) + 1)

# persons1 - df1
persons1 = df1[['id_person', 'Gender', 'Age', 'family_history_with_overweight', 'Height', 'Weight', 'NObeyesdad']].rename(
    columns={'Gender': 'gender', 'Age': 'age', 'family_history_with_overweight': 'health_history', 'Height': 'height', 'Weight': 'weight', 'NObeyesdad':'obesity_level'}
)
print("persons1 préparé.")

persons1 préparé.


In [27]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Charger les variables d'environnement et créer une connexion à la base de données
def get_db_engine():
    load_dotenv()
    db_config = {
        'host': os.getenv('DB_HOST'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'database': os.getenv('DB_NAME')
    }
    connection_url = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
    return create_engine(connection_url)

# Connexion à la base de données
engine = get_db_engine()

# Charger la table Is_Smoker depuis la base de données
is_smoker_table = pd.read_sql("SELECT * FROM Is_Smoker", con=engine)
print("Table Is_Smoker chargée :")
print(is_smoker_table)

# Charger le fichier CSV
file1 = 'data/ObesityDataSet_raw_and_data_sinthetic.csv'
df1 = pd.read_csv(file1)

# Vérifier les valeurs uniques de la colonne SMOKE
print("Valeurs uniques dans la colonne SMOKE :")
print(df1['SMOKE'].unique())

# Convertir SMOKE en booléen basé sur les valeurs "no" et "yes"
df1['SMOKE'] = df1['SMOKE'].map({'no': False, 'yes': True})

# Ajouter une colonne id_smoker dans df1 en fonction de la valeur de SMOKE
def map_smoker_id(smoke_value):
    smoker_row = is_smoker_table[is_smoker_table['smoker'] == smoke_value]
    if not smoker_row.empty:
        return smoker_row.iloc[0]['id_smoker']
    return None

df1['id_smoker'] = df1['SMOKE'].apply(map_smoker_id)

# Vérifier que id_smoker a été correctement mappé
print("Aperçu des colonnes SMOKE et id_smoker :")
print(df1[['SMOKE', 'id_smoker']].head())

# Créer le DataFrame Health_FACT
health_fact = pd.DataFrame({
    'id_person': df1.index + 1,  # ID des personnes basé sur l'index du fichier df1
    'id_habits': 1,  # Placeholder, à mapper selon d'autres données
    'id_drinks': 1,  # Placeholder, à mapper selon d'autres données
    'id_foods': 1,  # Placeholder, à mapper selon d'autres données
    'id_diet': 1,  # Placeholder, à mapper selon d'autres données
    'id_smoker': df1['id_smoker'],
    'id_env': 1,  # Placeholder, à mapper selon d'autres données
    'id_sport': 1,  # Placeholder, à mapper selon d'autres données
    'health_score': None,  # Placeholder, à mapper selon d'autres données
    'ratio_cal': None  # Placeholder, à mapper selon d'autres données
})

# Afficher un aperçu pour validation
print("Aperçu de la table Health_FACT :")
health_fact.head()


Table Is_Smoker chargée :
   id_smoker  smoker
0          1       0
1          2       1
Valeurs uniques dans la colonne SMOKE :
['no' 'yes']
Aperçu des colonnes SMOKE et id_smoker :
   SMOKE  id_smoker
0  False          1
1   True          2
2  False          1
3  False          1
4  False          1
Aperçu de la table Health_FACT :


,id_person,id_habits,id_drinks,id_foods,id_diet,id_smoker,id_env,id_sport,health_score,ratio_cal
0,1,1,1,1,1,1,1,1,None,None
1,2,1,1,1,1,2,1,1,None,None
2,3,1,1,1,1,1,1,1,None,None
3,4,1,1,1,1,1,1,1,None,None
4,5,1,1,1,1,1,1,1,None,None


In [29]:
# Insérer les données dans la table Health_FACT
health_fact.to_sql('Health_FACT', con=engine, if_exists='append', index=False)
print("Données insérées dans la table Health_FACT.")

Données insérées dans la table Health_FACT.


C:\Users\user\AppData\Local\Temp\ipykernel_5556\1246365194.py:2: UserWarning: The provided table name 'Health_FACT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  health_fact.to_sql('Health_FACT', con=engine, if_exists='append', index=False)
